In [3]:
import pandas as pd

In [9]:
first_part = pd.read_csv('Data/Covid-19 Twitter Dataset (Apr-Jun 2020).csv')
second_part = pd.read_csv('Data/Covid-19 Twitter Dataset (Apr-Jun 2021).csv')
third_part = pd.read_csv('Data/Covid-19 Twitter Dataset (Aug-Sep 2020).csv')


data = pd.concat([first_part, second_part, third_part])

,id,created_at,source,original_text,lang,favorite_count,retweet_count,original_author,hashtags,user_mentions,place,clean_tweet,compound,neg,neu,pos,sentiment
0,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @GlblCtzn: .@priyankachopra is calling on l...,en,0.0,31.0,RJIshak,NaN,"GlblCtzn, priyankachopra",Jakarta Capital Region,call leader help protect refuge covid19 provid...,0.8176,0.000,0.452,0.548,pos
1,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,en,0.0,61.0,makinwaoluwole,NaN,OGSG_Official,Nigeria,ogun state support cbn nirsal covid19 target c...,0.6486,0.000,0.602,0.398,pos
2,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @AdvoBarryRoux: These 5 police officials ba...,en,0.0,1.0,TembeAmu,NaN,AdvoBarryRoux,NaN,polic offici base namahadi polic station busi ...,0.2732,0.000,0.851,0.149,pos
3,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @MobilePunch: COVID-19: Oyo discharges two ...,en,0.0,0.0,ilyasrabiu,NaN,MobilePunch,"Lagos, Nigeria",covid19 oyo discharg two patient,0.0000,0.000,1.000,0.000,neu
4,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",My Condolences to the Family of those who did ...,en,0.0,13869.0,bucketeconomist,Covid_19,NaN,NaN,condol famili surviv,0.0000,0.000,1.000,0.000,neu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120504,1.320000e+18,2020-10-20,"<a href=""https://about.twitter.com/products/tw...",RT @Reuters: Exclusive: AstraZeneca U.S. COVID...,en,0.0,51.0,Sabahatjahan28,NaN,Reuters,India,exclus astrazeneca covid19 vaccin trial may re...,0.0000,0.000,1.000,0.000,neu
120505,1.320000e+18,2020-10-20,"<a href=""http://twitter.com/#!/download/ipad"" ...","RT @AndrewTSaks: 5,000+ workers from 130+ coun...",en,0.0,36.0,jefwintermans,NaN,AndrewTSaks,"Den Haag, Nederland",worker countri sign petit reinstat worker mass...,-0.3400,0.211,0.789,0.000,neg
120506,1.320000e+18,2020-10-20,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Culture of China-Brilliance of Huaxing\nCelebr...,en,0.0,9.0,ChinaEmbOttawa,NaN,NaN,"515 St. Patrick Street, Ottawa",cultur china brillianc huax celebr th annivers...,-0.3818,0.191,0.809,0.000,neg
120507,1.320000e+18,2020-10-20,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Trump called CNN 'bastards' for covering Covid...,en,0.0,31451.0,maetsfollufdaeh,NaN,NaN,NaN,trump call cnn bastard cover covid19 hear keil...,-0.5423,0.304,0.696,0.000,neg
